In [204]:
import gradio as gr
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier


# Load your data
data = pd.read_csv('dataset_part_2.csv')
features = data[['PayloadMass', 'Orbit', 'LaunchSite']]
Y = data['Class'].to_numpy()

# Define preprocessor with one-hot encoding
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), ['PayloadMass']),
        ('cat', OneHotEncoder(drop='first', handle_unknown='ignore'), ['Orbit', 'LaunchSite'])
    ]
)
# Best parameters: {'classifier__max_depth': None, 'classifier__min_samples_leaf': 2, 'classifier__min_samples_split': 5}

# Create a pipeline
# pipeline = Pipeline(steps=[
#     ('preprocessor', preprocessor),
#     ('classifier', DecisionTreeClassifier(
#         ccp_alpha=0.0194,
#         class_weight='balanced',
#         criterion='gini',
#         max_depth=None,
#         max_features='sqrt',
#         min_samples_leaf=2,
#         min_samples_split=5,
#         splitter='random'
#     ))
# ])
# Create a pipeline
pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', RandomForestClassifier(class_weight='balanced', random_state=2)
    )])
# Split the dataset
X_train, X_test, Y_train, Y_test = train_test_split(features, Y, test_size=0.2, random_state=2, stratify=Y)

# Fit the model
pipeline.fit(X_train, Y_train)

def predict_class(mass_payload, orbit, launch_site):
    # Prepare input data as a DataFrame
    input_data = pd.DataFrame([[mass_payload, orbit, launch_site]], 
                               columns=['PayloadMass', 'Orbit', 'LaunchSite'])
    
    # Make prediction using the pipeline
    prediction = pipeline.predict(input_data)
    
    return "Success" if prediction[0] == 1 else "Failure"

# Gradio inputs
mass_payload_input = gr.Number(label='Mass Payload (kg)', value=5000)
orbit_input = gr.Dropdown(choices=['LEO', 'GTO', 'ISS', 'PO', 'VLEO', 'SSO', 'GEO', 'MEO', 'HEO', 'ES-L1'], label='Orbit')
launch_site_input = gr.Dropdown(choices=['KSC LC 39A', 'CCSFS SLC 40', 'VAFB SLC 4E'], label='Launch Site')

# Gradio interface
interface = gr.Interface(
    fn=predict_class, 
    inputs=[mass_payload_input, orbit_input, launch_site_input], 
    outputs="text", 
    title="Space Y Mission Success Predictor",
    description="Predict the success or failure of a space mission based on Mass Payload, Orbit, and Launch Site."
)

# Launch the interface
interface.launch(share=True)  # Use share=True for remote access


Running on local URL:  http://127.0.0.1:7911

Could not create share link. Please check your internet connection or our status page: https://status.gradio.app.


In [197]:
from sklearn.metrics import classification_report

# After fitting the model, check the performance on the test set
y_pred = pipeline.predict(X_test)
print(classification_report(Y_test, y_pred))

              precision    recall  f1-score   support

           0       0.33      1.00      0.50         6
           1       0.00      0.00      0.00        12

    accuracy                           0.33        18
   macro avg       0.17      0.50      0.25        18
weighted avg       0.11      0.33      0.17        18



D:\programs\Anaconda3\envs\gradio_env\lib\site-packages\sklearn\preprocessing\_encoders.py:227: UserWarning: Found unknown categories in columns [0] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
D:\programs\Anaconda3\envs\gradio_env\lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
D:\programs\Anaconda3\envs\gradio_env\lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
D:\programs\Anaconda3\envs\gradio_env\lib\site-packages\sklearn\metrics\_classification.py:1469: Undefined

In [ ]:
from sklearn.model_selection import GridSearchCV

param_grid = {
    'classifier__max_depth': [None, 10, 20, 30,40,50,60],
    'classifier__min_samples_split': [2, 5, 10,15],
    'classifier__min_samples_leaf': [1, 2, 4,6]
}

grid_search = GridSearchCV(pipeline, param_grid, cv=5, scoring='accuracy')
grid_search.fit(X_train, Y_train)
print("Best parameters:", grid_search.best_params_)

In [201]:
from sklearn.ensemble import RandomForestClassifier


# Create a pipeline
pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', RandomForestClassifier(class_weight='balanced', random_state=2)
    )])
pipeline.fit(X_train, Y_train)
rf_pred = pipeline.predict(X_test)

print(classification_report(Y_test, rf_pred))


              precision    recall  f1-score   support

           0       0.38      0.50      0.43         6
           1       0.70      0.58      0.64        12

    accuracy                           0.56        18
   macro avg       0.54      0.54      0.53        18
weighted avg       0.59      0.56      0.57        18



D:\programs\Anaconda3\envs\gradio_env\lib\site-packages\sklearn\preprocessing\_encoders.py:227: UserWarning: Found unknown categories in columns [0] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(


In [203]:
predictions = pd.DataFrame({'Actual': Y_test, 'Predicted': rf_pred})
print(predictions.head(20))  # View the first 20 predictions

    Actual  Predicted
0        0          0
1        1          1
2        1          1
3        1          0
4        0          1
5        1          0
6        1          0
7        1          0
8        0          0
9        1          0
10       1          1
11       1          1
12       0          1
13       0          0
14       1          1
15       1          1
16       0          1
17       1          1


In [207]:
# Example input
test_input = pd.DataFrame([[6123.54764705882, 'LEO', 'CCSFS SLC 40']], columns=['PayloadMass', 'Orbit', 'LaunchSite'])

# Get prediction
test_prediction = pipeline.predict(test_input)
print("Test Prediction:", "Success" if test_prediction[0] == 1 else "Failure")

Test Prediction: Failure
